In [ ]:
import sys
from pathlib import Path
sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork
import pandas as pd
import geopandas as gpd
from numba.core.errors import NumbaDeprecationWarning
import warnings
warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

Define base directory and results directory

In [ ]:
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")

Define network name and load areas (discharge units: afwaterende eenheden)

In [ ]:
network_name = "zutphen_tki_netwerk"

areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)

Create networkanalysis

In [ ]:
network = RibasimLumpingNetwork(name=network_name, areas_gdf=areas_gdf)

Select simulation sets and extract all data points

In [ ]:
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)

In [ ]:
network.get_network_data()

Define node_ids on which to split the network into Ribasim basins:
- define types to include
- define additional split nodes by id
- define which of the node_ids should be excluded
- combine types of split_node_ids
- add split_nodes to network

In [ ]:
network.add_split_nodes_based_on_node_ids(
    weirs=True,
    pumps=True,
    bifurcations=False,
    confluences=False,
    split_node_ids_to_include=[1444, 1448, 1377, 2378, 419, 1183],
    split_node_ids_to_exclude=[314]
);

Create basins (gdf) base don nodes, edges, split_nodes and areas

In [ ]:
network.create_basins_based_on_split_nodes();

Export everything to geopackage

In [ ]:
network.export_to_geopackage(output_dir=results_dir)

DATA OVERVIEW

In [ ]:
network.nodes_gdf.head()

In [ ]:
network.edges_gdf.head()

In [ ]:
network.split_nodes.head()

In [ ]:
network.basins_gdf.head()

In [ ]:
network.basin_areas_gdf.head()

In [ ]:
network.split_nodes